## 0. 준비

### 1) 라이브러리 설치

In [ ]:
import os
os.getcwd()


'/content'

In [ ]:
!pip install wandb

In [ ]:
 #!wandb --help

In [ ]:
#!wandb login #--relogin
# kkukky@naver.com  220215bca12e71dfd5815f1648ca8dbbb2c1bef8
#  17b70d1b235684f485db5bcc4b47788ca0e90fd2
import wandb
wandb.login(key="220215bca12e71dfd5815f1648ca8dbbb2c1bef8")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install wandb
!pip install torch
!pip install pytorch_lightning
!pip install rouge

In [ ]:

import pandas as pd             # 데이터 프레임을 다루기 위한 라이브러리입니다. 주로 데이터 처리 및 분석에 사용됩니다.
import os                       # 운영 체제와 상호작용하기 위한 모듈로, 파일 및 디렉터리 작업에 사용됩니다.
import re                       # 정규 표현식을 사용하여 문자열을 검색, 처리하는 데 사용됩니다.
import json                     # JSON 형식의 데이터를 처리하기 위한 라이브러리입니다.
import yaml                     # YAML 형식의 데이터를 처리하기 위한 라이브러리입니다.
from glob import glob           # 특정 패턴에 맞는 파일 경로들을 리스트로 반환하는 모듈입니다.
from tqdm import tqdm           # 반복문에 대한 진행 상황을 시각적으로 보여주는 라이브러리입니다.
from pprint import pprint       # 데이터를 좀 더 읽기 쉽게 출력하기 위한 라이브러리입니다.
import torch                    # PyTorch 라이브러리로, 딥러닝 모델을 구축하고 학습하기 위한 핵심 라이브러리입니다.
import pytorch_lightning as pl  # PyTorch의 고수준 API로, 모델 학습을 간소화하고 구조화된 방식으로 진행할 수 있습니다.
from rouge import Rouge         # 텍스트 요약 및 생성 모델의 성능을 평가하기 위해 사용하는 지표 중 하나입니다.

from torch.utils.data import Dataset, DataLoader  # 데이터셋을 다루고, 이를 모델 학습에 사용할 수 있도록 배치(batch) 단위로 나누는 데 사용됩니다.
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig  # 트랜스포머 모델을 위한 라이브러리로, 토크나이저와 모델을 불러오는 데 사용됩니다.
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer  # Seq2Seq (Sequence-to-Sequence) 모델 학습을 위한 도구와 설정을 제공합니다.
from transformers import Trainer, TrainingArguments  # 일반적인 모델 학습을 위한 도구와 설정을 제공합니다.
from transformers import EarlyStoppingCallback  # 학습 과정에서 성능 향상이 없을 때 조기 종료를 할 수 있도록 도와주는 콜백 함수입니다.

import wandb                    # 모델 학습 과정을 쉽게 추적하고 시각화할 수 있는 툴입니다. 주로 실험 관리 및 결과 기록에 사용됩니다.

#!wandb login #--relogin
# kkukky@naver.com     220215bca12e71dfd5815f1648ca8dbbb2c1bef8
# kkukky81@gmail.com   17b70d1b235684f485db5bcc4b47788ca0e90fd2



### 2) 모델 선택, title 만들기

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

### bart 계열
# model = "digit82/kobart-summarization"
# model = "NLPBada/kobart-chat-persona-extraction-v2"
# model = "EbanLee/kobart-summary-v3"
model = 'jx7789/kobart_summary_v3'
# model = 'gogamza/kobart-summarization'

### T5 계열
# model = "t5-small"
# model = "KETI-AIR/ke-t5-base-ko" 값이 안나옴
# model = "eenzeenee/t5-base-korean-summarization" 안됨
# model = 'psyche/KoT5-summarization' #안됨
# model = 'csebuetnlp/mT5_multilingual_XLSum'


train = "train_3x5075p.csv"
para = "3x5075p-l6-n4-b5"


title = model.replace('/','_') + para

# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
#tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
tokenizer = AutoTokenizer.from_pretrained(model)

#from transformers import AutoModel

#model = AutoModel.from_pretrained(model, force_download=True)

print('###   ', title, '   ###')

#output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)


###    jx7789_kobart_summary_v33x5075p-l6-n4-b5    ###


### 3) config 파일 만들기

In [ ]:
batch = 16
config_data = {
    "general": {
        "data_path": "/content/",  # 모델 학습에 사용할 데이터가 저장된 경로를 지정합니다.
        #"model_name": "digit82/kobart-summarization",  # 사용할 사전 학습된 모델의 이름을 지정합니다.
        "model_name": model,  # 사용할 사전 학습된 모델의 이름을 지정합니다.
        "output_dir": "/content/"  # 모델의 출력물(예: 생성된 텍스트)을 저장할 디렉터리를 지정합니다.
    },
    "training": {
        "overwrite_output_dir": True,       # True로 설정하면, 기존에 존재하는 출력 디렉터리 내용을 덮어씁니다.
        "num_train_epochs": 20,             # 전체 데이터셋을 몇 번 반복해서 학습할지를 설정합니다. (20번)
        "learning_rate": 1e-6,              # 학습률(learning rate)을 설정합니다.
        "per_device_train_batch_size": batch,  # 각 디바이스(예: GPU)에서 한 번에 학습할 데이터 배치의 크기를 설정합니다. (50)
        "per_device_eval_batch_size": batch,   # 평가 시 사용할 배치 크기를 설정합니다. (32)
        "warmup_ratio": 0.1,                # 학습 초기에 학습률을 천천히 증가시키는 비율을 설정합니다.
        "weight_decay": 0.01,               # 가중치 감쇠(weight decay) 값을 설정합니다. (과적합 방지를 위해 사용)
        "lr_scheduler_type": 'cosine_with_restarts',      # 학습률 스케줄러 유형을 'cosine'으로 설정합니다.
        "optim": 'adamw_torch',             # 옵티마이저(optimizer)로 AdamW를 사용합니다.
        "gradient_accumulation_steps": 1,   # 기울기(gradient) 축적을 위한 스텝 수를 설정합니다. (1이면 축적 없이 바로 업데이트)
        "evaluation_strategy": 'epoch',     # 평가를 언제 수행할지 설정합니다. ('epoch'는 각 에폭 종료 시 평가)
        "save_strategy": 'epoch',           # 모델을 저장할 시점을 설정합니다. ('epoch'는 각 에폭 종료 시 저장)
        "save_total_limit": 7,              # 최대 저장할 체크포인트 수를 설정합니다. (가장 최근 7개만 유지)
        "fp16": True,                       # 반정밀도(floating point 16) 연산을 사용할지 설정합니다. (True이면 메모리 절약 및 속도 향상)
        "load_best_model_at_end": True,     # 학습이 종료될 때 가장 성능이 좋은 모델을 불러옵니다.
        "seed": 42,                         # 랜덤 시드를 설정하여 실험의 재현성을 보장합니다.
        "logging_dir": "./logs",            # 학습 로그를 저장할 디렉터리를 설정합니다.
        "logging_strategy": "epoch",        # 로그를 언제 기록할지 설정합니다. ('epoch'는 각 에폭 종료 시 기록)
        "predict_with_generate": True,      # 평가 시 텍스트 생성을 수행할지 설정합니다.
        "generation_max_length": 200,       # 텍스트 생성 시 최대 길이를 설정합니다. (100 토큰)
        "do_train": True,                   # 모델을 학습할지 여부를 설정합니다. (True로 설정)
        "do_eval": True,                    # 모델을 평가할지 여부를 설정합니다. (True로 설정)
        "early_stopping_patience": 3,       # 조기 종료를 위한 인내 기간(몇 번의 에폭 동안 성능 개선이 없으면 종료)을 설정합니다. (3)
        "early_stopping_threshold": 0.001,  # 조기 종료를 위한 성능 개선 최소 임계값을 설정합니다. (0.001)
        "report_to": "wandb"                # (선택 사항) wandb를 사용하여 학습 과정을 보고할지 설정합니다.
    },
    "inference": {
        "ckt_path": "/content/code/",  # 학습된 모델의 체크포인트 파일 경로를 설정합니다.
        "result_path": "/content/prediction/",  # 추론 결과를 저장할 경로를 설정합니다.
        "no_repeat_ngram_size": 4,  # 생성된 텍스트에서 동일한 n-gram이 반복되지 않도록 설정합니다. (2-gram 기준)
        "early_stopping": True,  # 조기 종료를 사용할지 여부를 설정합니다.
        "generate_max_length": 200,  # 생성 텍스트의 최대 길이를 설정합니다. (100 토큰)
        "num_beams": 5,  # 빔 서치(beam search) 시 사용할 빔 수를 설정합니다. 1~5까지 가능함. (4)
        "batch_size": batch,  # 추론 시 사용할 배치 크기를 설정합니다. (32)
        # 모델 생성 결과에서 불필요한 토큰들을 제거합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    },
    "tokenizer": {
        "encoder_max_len": 1024,  # 입력 텍스트를 인코딩할 때 최대 길이를 설정합니다. (512 토큰)
        "decoder_max_len": 200,  # 출력 텍스트(생성된 텍스트)를 디코딩할 때 최대 길이를 설정합니다. (100 토큰)
        "bos_token": f"{tokenizer.bos_token}",  # 시작 토큰(beginning of sentence)을 지정합니다.
        "eos_token": f"{tokenizer.eos_token}",  # 끝 토큰(end of sentence)을 지정합니다.
        # 특정 단어들이 분해되지 않도록, special_tokens을 지정하여 토크나이저에서 처리할 때 이들 단어를 그대로 유지합니다.
        "special_tokens": [ '#Person1#',     #76737
                            '#Person2#',     #70211
                            '#Person3#',     #452
                            '#Person4#',     #41
                            '#Person5#',     #5
                            '#Person6#',     #9
                            '#Person7#',     #3
                            '#SSN#',         #3
                            '#PhoneNumber#', #203
                            '#Address#',     #45
                            '#Email#',       #17
                            '#CarNumber#',   #6
                            '#CardNumber#'   #10
                            '#DateOfBirth#', #8
                            '#PassportNumber#']  #7
                            # #Person 2#    띄어쓰기 오타 1개 o
                            # 9547,9548     #Person1      #없는 오타 2개 o
                            # 9547,9548     #Person2      #없는 오타 2개 o
                            # 9750,9779     Person1#      #없는 오타 2개 o
                            # 420           #PhoneNumber  #없는 오타 1개 o
                            # #Person#      숫자 없는 오타 1개 o
                            # 839      #사람1만기 시 계정 갱신 o
                            # 1033sum  이 사람2#에게 내일 아침 o
                            # 1125     사람1#: 제니, 이번 o
                            # 1133     사람2#은 그 기간 동 o
                            # 1030     이 사람2#에게 내일 o
                            # 1142     사람1#: 실례합니다. 저는 o
                            # 1199     사람1#은 시험에 대한 준비가 된 o
                            # 1213     #하지만 장기간의 o
                            # 1236     #고객님, 크루즈 컨트롤에 o
                            # 1250     ##여기 있습니다. 스티븐 o
                            # 1266     #고객님, 저희는 고객이 화나 o
                            # 1278     #고객님, 죄송합니다만 계 o
                            # 1281     #잠깐만요, 버전 7 o
                            # 1283     #어디 보자. 네, 그런 방 o
                            # 1301     #샐러드용 드레싱은 o
                            # 1302     #페리에와 짐 빔 세 o
                            # 1306     #나 부엌에 있어. . . o
                            # 1322     #여기서 만나서 반갑 o
                            # 1547     #작은 걸로 주세 o
                            # 1609     #여기 있습니다. o
                            # 정상 8320   음표는 G#이라고 써있어.
                            # 10370    회사 #에서 기술자로 근  o
                            # 11716    ##: 안녕, 프란시스   #이 두개 o
                            # 4001     (Person A가 탈의실에서 나옴) 스웨터는 어떠셨나요? o
                            # 2255     전화번호는 610-555-1234입니다. o
                            # 2719     네, 488-6361입니다. 3시까지 저 o
                            # 2980     전화번호는 513-3284입니다. o
    },
    # (선택 사항) wandb 설정: wandb 홈페이지에서 받은 entity, project, run_name 정보를 설정합니다.
    "wandb": {
         "entity": "kkukky-empty",  # wandb에서 실험을 기록할 엔티티(entity)를 설정합니다.
         "project": "NLP2",  # wandb 프로젝트 이름을 설정합니다.
         "name": title # wandb에서 실행(run) 이름을 설정합니다.
    }
}


### 변수 적용 간편화를 위해 코드 옮김

# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config_bart.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

# 저장된 config 파일을 불러옵니다.
config_path = "./config_bart.yaml"
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
# pprint(config)


### 4) 데이터 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = config['general']['data_path']
print(data_path)

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,train))
train_df.tail()

/content/


FileNotFoundError: [Errno 2] No such file or directory: '/content/train_3x5070p.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

In [ ]:
# validation data의 구조와 내용을 확인합니다.
test_df = pd.read_csv(os.path.join(data_path,'test.csv'))
test_df.tail()

## 1. 모델 트레이닝

In [ ]:
# 사용할 device를 정의합니다. GPU가 사용 가능하면 'cuda:0', 그렇지 않으면 'cpu'를 사용합니다.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('-' * 10, f'device : {device}', '-' * 10,)
print(torch.__version__)

---------- device : cuda:0 ----------
2.4.0+cu121


### 1) 토크나이저와 모델 로드

In [ ]:
def load_tokenizer_and_model_for_train(config, device):
    print('-' * 10, 'Load tokenizer & model', '-' * 10,)
    print('-' * 10, f'Model Name : {config["general"]["model_name"]}', '-' * 10,)

     #모델 이름 불러오기:
    # config['general']['model_name']을 통해 사전 학습된 모델의 이름을 설정 파일에서 불러옵니다.
    # 이는 Hugging Face의 모델 허브에서 가져올 모델의 이름입니다.
    model_name = config['general']['model_name']


    ### BART ##################
    # BART 설정 로드:
    # BartConfig를 사용하여 지정된 모델 이름으로부터 BART의 설정을 불러옵니다.
    # 이 설정은 모델의 구조와 하이퍼파라미터를 정의합니다.
    bart_config = BartConfig().from_pretrained(model_name)

    # 토크나이저 로드:
    # AutoTokenizer.from_pretrained를 사용하여 지정된 모델 이름으로부터 토크나이저를 불러옵니다.
    # 이 토크나이저는 텍스트 데이터를 토큰화하는 역할을 합니다.
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 사전 학습된 모델 로드:
    # BartForConditionalGeneration.from_pretrained를 사용하여 사전 학습된 BART 모델을 불러옵니다.
    # 이 모델은 텍스트 생성, 요약 등의 작업을 수행하는 데 사용됩니다.
    generate_model = BartForConditionalGeneration.from_pretrained(model_name, config=bart_config)


    # 특수 토큰 추가:
    # special_tokens_dict를 사용하여 설정에서 정의한 특수 토큰을 토크나이저에 추가합니다.
    # 이는 예를 들어, 특정 인물이나 장소를 나타내는 토큰을 추가하는 등의 작업에 사용됩니다.
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)
    # 토크나이저에 추가된 토큰에 맞추어 모델의 토큰 임베딩 크기를 조정합니다.
    # 모델의 토큰 임베딩 크기를 업데이트된 토크나이저의 크기에 맞게 조정합니다.
    generate_model.resize_token_embeddings(len(tokenizer))


    # 모델을 지정된 디바이스로 이동:
    # 모델을 device 변수에 지정된 디바이스(GPU 또는 CPU)로 이동시킵니다.
    # 이를 통해 모델이 올바른 디바이스에서 실행되도록 합니다.
    generate_model.to(device)


    # 모델 설정 출력:
    # 로드된 모델의 설정을 출력하여, 모델의 구조와 하이퍼파라미터 등을 확인할 수 있습니다.
    print(generate_model.config)

    print('-' * 10, 'Load tokenizer & model complete', '-' * 10,)

    # 모델과 토크나이저 반환:
    return generate_model, tokenizer

    # 최종적으로 불러온 모델과 토크나이저를 반환하여, 이후 학습 과정에서 사용할 수 있도록 준비합니다.
    # 이 함수는 학습을 위해 필요한 모델과 토크나이저를 설정 파일에 따라 자동으로 불러오고 준비해 줍니다.
    # 특수 토큰의 추가 및 모델 임베딩의 재구성도 함께 처리하여, 모델이 주어진 작업에 최적화될 수 있도록 돕습니다.



In [ ]:
# 사용할 모델과 토크나이저를 로드합니다.
print("\n#######  사용할 모델과 토크나이저를 로드합니다.  ################################################################################")
generate_model, tokenizer = load_tokenizer_and_model_for_train(config, device)
print('-' * 10, "tokenizer special tokens : ", tokenizer.special_tokens_map, '-' * 10)


#######  사용할 모델과 토크나이저를 로드합니다.  ################################################################################
---------- Load tokenizer & model ----------
---------- Model Name : jx7789/kobart_summary_v3 ----------


config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

BartConfig {
  "_name_or_path": "jx7789/kobart_summary_v3",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_enco

### 2) 전처리

In [ ]:

# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        # 클래스 초기화 메서드입니다. 시작 토큰(bos_token)과 종료 토큰(eos_token)을 인스턴스 변수로 저장합니다.
        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 파일 경로를 입력 받아 CSV 파일을 읽고, 필요한 컬럼들만 선택하여 데이터프레임으로 반환합니다.
    def make_set_as_df(file_path, is_train = True):

        # is_train이 True면 학습용 데이터로, False면 테스트용 데이터로 처리합니다.
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue','summary']]
            return train_df

        # 테스트 데이터인 경우, 'fname'과 'dialogue' 컬럼만 선택하여 반환합니다.
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue']]
            return test_df


    # BART 모델의 입력과 출력을 생성하는 메서드입니다.
    def make_input(self, dataset, is_test=False):

        # is_test가 True면 테스트 데이터를 위한 입력만 생성하고, False면 학습 데이터를 위한 입력과 출력을 생성합니다.
        if is_test:
            # 테스트 데이터의 경우, 인코더 입력과 디코더의 시작 토큰으로만 구성된 입력을 반환합니다.
            # 인코더 입력으로 'dialogue' 텍스트를 사용합니다.
            encoder_input = dataset['dialogue']
            # 디코더 입력은 시작 토큰으로만 구성합니다.
            decoder_input = [self.bos_token] * len(dataset['dialogue'])

            return encoder_input.tolist(), list(decoder_input)
        else:
            # 학습 데이터의 경우, 인코더 입력, 디코더 입력, 디코더 출력을 모두 생성합니다.
            encoder_input = dataset['dialogue']  # 인코더 입력으로 'dialogue' 텍스트를 사용합니다.

            # 디코더 입력은 시작 토큰(bos_token)과 요약 텍스트(summary)로 구성됩니다.
            decoder_input = dataset['summary'].apply(lambda x: self.bos_token + str(x))

            # 디코더 출력은 요약 텍스트(summary)와 종료 토큰(eos_token)으로 구성됩니다.
            decoder_output = dataset['summary'].apply(lambda x: str(x) + self.eos_token)

            # 리스트로 변환하여 반환합니다.
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

In [ ]:
# 학습에 사용할 데이터셋을 전처리하고 로드합니다.
print("\n#######  학습에 사용할 데이터셋을 전처리하고 로드합니다.  ########################################################################")
# 시작 토큰(beginning of sentence)과 종료 토큰(end of sentence)을 설정합니다.
preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])



#######  학습에 사용할 데이터셋을 전처리하고 로드합니다.  ########################################################################


### 3) 학습 및 검증 데이터셋 준비

#### 3-1) Train, Validation, Test 클래스 정의

In [ ]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        # 학습 데이터셋 초기화 메서드입니다. 인코더 입력, 디코더 입력, 레이블, 데이터 길이를 저장합니다.
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터를 반환하는 메서드입니다.

        # 인코더 입력 데이터에서 해당 인덱스의 데이터를 복사하여 `item` 딕셔너리에 저장합니다.
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}  # item[input_ids], item[attention_mask]

        # 디코더 입력 데이터에서 해당 인덱스의 데이터를 복사하여 `item2` 딕셔너리에 저장합니다.
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()}  # item2[input_ids], item2[attention_mask]

        # `item2` 딕셔너리의 'input_ids'와 'attention_mask'를 각각 'decoder_input_ids'와 'decoder_attention_mask'로 이름을 변경합니다.
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')  # 'input_ids' 키 제거
        item2.pop('attention_mask')  # 'attention_mask' 키 제거

        # `item` 딕셔너리에 디코더의 입력 정보를 추가합니다.
        item.update(item2)  # item[input_ids], item[attention_mask], item[decoder_input_ids], item[decoder_attention_mask]

        # 레이블로 사용할 'input_ids' 값을 `item` 딕셔너리에 추가합니다.
        item['labels'] = self.labels['input_ids'][idx]  # item[input_ids], item[attention_mask], item[decoder_input_ids], item[decoder_attention_mask], item[labels]

        return item

    def __len__(self):
        # 데이터셋의 길이를 반환하는 메서드입니다.
        return self.len


# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        # 검증 데이터셋 초기화 메서드입니다. 학습 데이터셋과 동일한 구조로 정의됩니다.
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터를 반환하는 메서드입니다.
        # 학습 데이터셋과 동일하게 정의됩니다.
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        # 데이터셋의 길이를 반환하는 메서드입니다.
        return self.len


# Test에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForInference(Dataset):     # inference뜻 : 추론
    def __init__(self, encoder_input, test_id, len):
        # 테스트 데이터셋 초기화 메서드입니다. 인코더 입력, 테스트 ID, 데이터 길이를 저장합니다.
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.len = len

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터를 반환하는 메서드입니다.
        # 인코더 입력 데이터에서 해당 인덱스의 데이터를 복사하여 `item` 딕셔너리에 저장합니다.
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
        item['ID'] = self.test_id[idx]
        return item

    def __len__(self):
        # 데이터셋의 길이를 반환하는 메서드입니다.
        return self.len


#### 3-2) Train, Validation 데이터셋 준비

In [ ]:
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):

    ### 데이터 로드 및 변환:
    #학습 및 검증 데이터 파일을 읽어 데이터프레임으로 변환합니다.
    #데이터프레임으로 변환된 데이터에서 대화(dialogue)와 요약(summary) 텍스트를 각각 학습 입력과 라벨로 사용합니다.
    # 데이터셋 경로를 지정합니다.
    train_file_path = os.path.join(data_path, train)  # 학습 데이터 파일 경로
    val_file_path = os.path.join(data_path, 'dev.csv')  # 검증 데이터 파일 경로
    # 학습(train)과 검증(validation) 데이터셋을 데이터프레임으로 변환합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)  # 학습 데이터 로드
    val_data = preprocessor.make_set_as_df(val_file_path)  # 검증 데이터 로드
    # 로드된 학습 데이터와 라벨의 첫 번째 샘플을 출력합니다.
    print('-' * 150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')
    # 로드된 검증 데이터와 라벨의 첫 번째 샘플을 출력합니다.
    print('-' * 150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')


    ### 데이터 전처리:
    # Preprocess 클래스의 make_input 메서드를 사용하여 인코더 입력, 디코더 입력, 디코더 출력을 생성합니다.
    # 이 과정에서 BART 모델에 적합한 형태로 텍스트 데이터를 구성합니다.
    # 학습 데이터에 대해 인코더 입력, 디코더 입력, 디코더 출력을 생성합니다.
    encoder_input_train, decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    # 검증 데이터에 대해 인코더 입력, 디코더 입력, 디코더 출력을 생성합니다.
    encoder_input_val, decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-' * 10, 'Load data complete', '-' * 10)


    ### 토큰화:
    # 텍스트 데이터를 tokenizer를 사용하여 토큰화합니다. 토큰화 과정에서:
    # 텍스트를 토큰 ID로 변환합니다.
    # 필요한 경우, 패딩(padding=True), 최대 길이(max_length), 특수 토큰(add_special_tokens=True) 등을 추가합니다.
    # 반환된 데이터는 PyTorch 텐서(return_tensors="pt")로 반환됩니다.
    # 학습 데이터의 인코더 입력을 토크나이저로 토큰화합니다.
    tokenized_encoder_inputs = tokenizer(
        encoder_input_train, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False
    )
    # 학습 데이터의 디코더 입력을 토크나이저로 토큰화합니다.
    tokenized_decoder_inputs = tokenizer(
        decoder_input_train, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False
    )
    # 학습 데이터의 디코더 출력을 토크나이저로 토큰화합니다.
    tokenized_decoder_ouputs = tokenizer(
        decoder_output_train, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False
    )


    ### 데이터셋 생성:
    # 학습 데이터와 검증 데이터를 각각 DatasetForTrain과 DatasetForVal 클래스로 감싸 데이터셋 객체를 생성합니다.
    # 이 객체들은 PyTorch의 DataLoader와 함께 사용되어 모델 학습 및 검증에 활용됩니다.
    # 학습용 데이터셋을 생성합니다.
    train_inputs_dataset = DatasetForTrain(
        tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs, len(encoder_input_train)
    )
    # 검증 데이터의 인코더 입력을 토크나이저로 토큰화합니다.
    val_tokenized_encoder_inputs = tokenizer(
        encoder_input_val, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False
    )
    # 검증 데이터의 디코더 입력을 토크나이저로 토큰화합니다.
    val_tokenized_decoder_inputs = tokenizer(
        decoder_input_val, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False
    )
    # 검증 데이터의 디코더 출력을 토크나이저로 토큰화합니다.
    val_tokenized_decoder_ouputs = tokenizer(
        decoder_output_val, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False
    )
    # 검증용 데이터셋을 생성합니다.
    val_inputs_dataset = DatasetForVal(
        val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs, len(encoder_input_val)
    )
    print('-' * 10, 'Make dataset complete', '-' * 10)


    ### 출력 및 반환:
    # 학습용 데이터셋과 검증용 데이터셋을 반환합니다. 이를 통해 모델 학습 과정에서 필요한 데이터를 제공하게 됩니다.
    return train_inputs_dataset, val_inputs_dataset


    # 이 함수는 모델 학습 및 검증을 위한 데이터 준비 과정에서 필요한
    # 모든 전처리, 토큰화, 데이터셋 생성을 자동으로 처리하여, 최종적으로 Dataset 객체를 반환합니다.
    # 이를 통해 모델 학습 및 검증을 효율적으로 수행할 수 있습니다.


In [ ]:
print("\n#######  학습 및 검증 데이터셋을 준비합니다.  ###################################################################################")
# 학습 및 검증 데이터셋을 준비합니다.
data_path = config['general']['data_path']  # 데이터 경로를 설정에서 가져옵니다.
train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config, preprocessor, data_path, tokenizer)



#######  학습 및 검증 데이터셋을 준비합니다.  ###################################################################################
------------------------------------------------------------------------------------------------------------------------------------------------------
train_data:
 #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.
train_label:
 스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.
----------------------

### 4) Trainer 클래스 초기화

In [ ]:
def compute_metrics(config, tokenizer, pred):
    ### rouge = Rouge():
    # ROUGE 점수를 계산하기 위해 Rouge 클래스를 초기화합니다.
    # ROUGE는 주로 텍스트 요약의 품질을 평가할 때 사용되는 지표입니다.
    rouge = Rouge()


    ### pred.predictions 및 pred.label_ids:
    # predictions: 모델이 예측한 토큰 ID의 배열입니다.
    # label_ids: 실제 레이블(정답) 토큰 ID의 배열입니다.
    # 예측된 토큰 ID와 실제 레이블 ID를 가져옵니다.
    predictions = pred.predictions
    labels = pred.label_ids


    ### 패딩 토큰 처리:
    # 예측 값과 레이블에서 -100으로 표시된 패딩 토큰을 실제 패딩 토큰 ID로 교체하여 평가에서 패딩이 영향을 미치지 않도록 합니다.
    # 모델 출력 중 패딩 토큰을 의미하는 -100 값을 tokenizer의 패딩 토큰 ID로 변경합니다.
    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id


    ### 토큰 디코딩:
    # 토큰 ID 배열을 원래의 텍스트 문자열로 변환합니다.
    # batch_decode는 여러 개의 토큰 배열을 한꺼번에 디코딩합니다.
    # 예측된 토큰 ID를 텍스트로 디코딩합니다.
    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    # 실제 레이블의 토큰 ID도 텍스트로 디코딩합니다.
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)


    ### 불필요한 토큰 제거:
    # 모델이 생성한 텍스트에서 사전에 정의된 불필요한 토큰을 제거하여 평가의 정확성을 높입니다.
    # 평가를 위해 불필요한 토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()  # 예측된 텍스트 복사
    replaced_labels = labels.copy()  # 실제 레이블 텍스트 복사
    remove_tokens = config['inference']['remove_tokens']  # 제거할 토큰 목록을 config에서 가져옵니다.
    # 각 불필요한 토큰을 제거합니다.
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token, " ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token, " ") for sentence in replaced_labels]


    ### 출력:
    # 평가를 위해 디코딩된 예측 텍스트와 실제 레이블의 일부 샘플을 출력합니다.
    # 첫 번째, 두 번째, 세 번째 예측과 실제 레이블을 출력합니다.
    print('-' * 150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-' * 150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-' * 150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")


    ### ROUGE 점수 계산:
    # replaced_predictions와 replaced_labels를 사용하여 ROUGE 점수를 계산합니다.
    # ROUGE-1, ROUGE-2, ROUGE-L 등의 F1 점수를 계산하여 반환합니다.
    # 최종적으로 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels, avg=True)


    ### 결과 반환:
    # ROUGE 점수 중 F1-score를 추출하여 딕셔너리 형태로 반환합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result


    # 이 함수는 모델이 생성한 텍스트의 품질을 ROUGE 지표로 평가하여,
    # 모델 성능을 평가하는 데 중요한 역할을 합니다.
    # ROUGE 점수를 통해 텍스트 요약 또는 생성 모델의 정확성을 평가할 수 있습니다.


In [ ]:
def load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset):
    print('-' * 10, 'Make training arguments', '-' * 10,)

    ### Seq2SeqTrainingArguments 설정:
    # 학습과 관련된 다양한 설정값들을 정의하는 Seq2SeqTrainingArguments 객체를 생성합니다.
    # 학습률, 배치 크기, 에포크 수, 로그 저장 위치 등 다양한 하이퍼파라미터와 옵션들이 포함됩니다.
    # 학습을 위한 설정값들을 정의합니다.
    training_args = Seq2SeqTrainingArguments(
        output_dir=config['general']['output_dir'],                                     # 모델 출력 디렉터리
        overwrite_output_dir=config['training']['overwrite_output_dir'],                # 출력 디렉터리를 덮어쓸지 여부
        num_train_epochs=config['training']['num_train_epochs'],                        # 전체 학습 에포크 수
        learning_rate=config['training']['learning_rate'],                              # 학습률
        per_device_train_batch_size=config['training']['per_device_train_batch_size'],  # 학습 시 디바이스당 배치 크기
        per_device_eval_batch_size=config['training']['per_device_eval_batch_size'],    # 평가 시 디바이스당 배치 크기
        warmup_ratio=config['training']['warmup_ratio'],                                # 학습 초기에 학습률을 점진적으로 증가시키는 비율
        weight_decay=config['training']['weight_decay'],                                # 가중치 감쇠 (과적합 방지)
        lr_scheduler_type=config['training']['lr_scheduler_type'],                      # 학습률 스케줄러 유형
        optim=config['training']['optim'],                                              # 옵티마이저 종류
        gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],  # 기울기(gradient) 축적 단계 수
        evaluation_strategy=config['training']['evaluation_strategy'],                  # 학습 중 평가 전략 (예: 에포크마다 평가)
        save_strategy=config['training']['save_strategy'],                              # 모델 저장 전략
        save_total_limit=config['training']['save_total_limit'],                        # 저장할 체크포인트의 최대 개수
        fp16=config['training']['fp16'],                                                # 반정밀도(float16) 연산 사용 여부
        load_best_model_at_end=config['training']['load_best_model_at_end'],            # 학습 종료 시 가장 좋은 모델 로드
        seed=config['training']['seed'],                                                # 랜덤 시드 설정
        logging_dir=config['training']['logging_dir'],                                  # 로그 저장 디렉터리
        logging_strategy=config['training']['logging_strategy'],                        # 로그 기록 전략 (예: 에포크마다 기록)
        predict_with_generate=config['training']['predict_with_generate'],              # 텍스트 생성 후 평가 지표를 계산할지 여부
        generation_max_length=config['training']['generation_max_length'],              # 텍스트 생성 시 최대 길이
        do_train=config['training']['do_train'],                                        # 학습 수행 여부
        do_eval=config['training']['do_eval'],                                          # 평가 수행 여부
        report_to=config['training']['report_to']                                       # (선택 사항) wandb로 학습 과정을 기록할지 여부
    )


    ### wandb 초기화 (선택 사항):
    # (선택 사항) wandb를 사용하여 학습 과정을 추적할 때 초기화합니다.
    # WandB(Weights & Biases)로 학습 과정을 추적하고 시각화하려면 wandb.init()을 사용해 초기화할 수 있습니다.
    # 이 부분은 현재 주석 처리되어 있으며, 필요한 경우 활성화할 수 있습니다.
    wandb.init(
         entity=config['wandb']['entity'],
         project=config['wandb']['project'],
         name=config['wandb']['name']
    )
    # (선택 사항) 모델 체크포인트를 wandb에 저장하도록 환경 변수를 설정합니다.
    os.environ["WANDB_LOG_MODEL"] = "checkpoint"
    os.environ["WANDB_WATCH"] = "false"


    ### EarlyStoppingCallback 설정:
    # EarlyStoppingCallback을 사용하여 학습이 진행될 때 검증 손실이 더 이상 개선되지 않으면 학습을 조기에 중단시킵니다.
    # 설정된 early_stopping_patience와 early_stopping_threshold에 따라 작동합니다.
    # EarlyStoppingCallback: 검증 손실이 더 이상 개선되지 않을 때 학습을 중단시키는 콜백을 설정합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],      # 개선이 없을 경우 중단까지 기다릴 에포크 수
        early_stopping_threshold=config['training']['early_stopping_threshold']     # 개선으로 간주할 최소 손실 감소량
    )
    print('-' * 10, 'Make training arguments complete', '-' * 10,)
    print('-' * 10, 'Make trainer', '-' * 10,)


    ### Seq2SeqTrainer 초기화:
    # Seq2SeqTrainer는 Hugging Face의 transformers 라이브러리에서 제공하는 훈련 도구로,
    # 시퀀스-투-시퀀스 모델의 학습과 평가를 위한 도구입니다.
    # 학습할 모델, 설정값, 학습/검증 데이터셋, 평가 메트릭 함수, 콜백 등을 인자로 받아 초기화합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model,  # 학습할 모델
        args=training_args,  # 학습 설정값
        train_dataset=train_inputs_dataset,  # 학습 데이터셋
        eval_dataset=val_inputs_dataset,  # 검증 데이터셋
        compute_metrics=lambda pred: compute_metrics(config, tokenizer, pred),  # 성능 평가를 위한 메트릭 함수
        callbacks=[MyCallback]  # EarlyStoppingCallback을 포함한 콜백 리스트
    )
    print('-' * 10, 'Make trainer complete', '-' * 10,)

    ### Trainer 객체 반환:
    # 생성된 trainer 객체를 반환하여, 이후 학습을 진행할 수 있게 합니다.
    return trainer


    # 이 함수는 모델 학습을 시작하기 위한 모든 준비 작업을 자동으로 처리하며,
    # 사용자에게 최적화된 Trainer 객체를 제공합니다. 이를 통해 학습 및 평가를 손쉽게 수행할 수 있습니다.


In [ ]:
print("\n#######  학습을 위한 Trainer 클래스를 초기화합니다.  ############################################################################")
# 학습을 위한 Trainer 클래스를 초기화합니다.
trainer = load_trainer_for_train(config, generate_model, tokenizer, train_inputs_dataset, val_inputs_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: Currently logged in as: kkukky (kkukky-empty). Use `wandb login --relogin` to force relogin



#######  학습을 위한 Trainer 클래스를 초기화합니다.  ############################################################################
---------- Make training arguments ----------


---------- Make training arguments complete ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


### 5) 모델 학습, wandb 세션종료

In [ ]:
print("\n#######  모델 학습을 시작합니다.  ##############################################################################################")
print(title)
# 모델 학습을 시작합니다.
trainer.train()

print("\n#######  wandb 세션을 종료합니다.  ############################################################################################")
# (선택) 모델 학습이 완료된 후 wandb 세션을 종료합니다.
wandb.finish()


#######  모델 학습을 시작합니다.  ##############################################################################################
jx7789_kobart_summary_v33x5075p-l6-n4-b5


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,5.969200,2.295457,0.183847,0.033186,0.179350


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   #Person1#은 숨을 쉴 때마다 가슴이 무겁게 느껴져 숨쉬기가 힘들다고 말한다. #Peron2#는 숨 쉬기가 힘들다는 것을 알고 있는 알레르기는 없다고 한다. #Pesson2 #는 숨 쉴 때마다 숨이 무겁다고 느낀다.                                                                                                                                            
GOLD: #Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다.                                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   #Person1#은 헤이, 헤이, 지미 때문에 다리와 팔목을 운동하러 가자고 제안한다. 3시 30분에 헬스장에서 만나기로 한다.                                                                                                                                                                      
GOLD: #Person1#은 

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 5, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 1}
wandb: Adding directory to artifact (/content/checkpoint-2336)... Done. 6.1s


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 5, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 1}
wandb: Adding directory to artifact (/content/checkpoint-23360)... Done. 6.4s


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   #Person2#는 숨을 쉴 때마다 가슴이 무겁고 숨쉬기가 힘들다고 말합니다. 의사는 천식에 대한 검사를 받게 할 예정입니다.                                                                 
GOLD: #Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다.                                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   지미는 #Person1#에게 헬스장에서 운동하자고 제안한다. # Person2#는 주간 스케줄을 따르고 있기 때문에 두 날을 바꾸기로 한다.                                                             
GOLD: #Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                                              
-----------------------------------------------------------------------------------------------

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 5, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 1}
wandb: Adding directory to artifact (/content/checkpoint-25696)... Done. 6.8s


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   #Person2#는 숨을 쉴 때마다 가슴이 무겁고 숨쉬기가 힘들다고 말합니다. 의사는 천식에 대한 검사를 받게 할 예정입니다.                                                                           
GOLD: #Person2#는 숨쉬기에 어려움을 겪는다. 의사는 #Person1#에게 이에 대해 묻고, #Person2#를 폐 전문의에게 보낼 예정이다.                                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:   지미는 #Person1#에게 운동하러 가자고 제안한다. 그들은 금요일에 다리와 팔목을 운동하기로 결정한다.                                                                                
GOLD: #Person1#은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                                              
-------------------------------------------------------------------------------------

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 5, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 1}
wandb: Adding directory to artifact (/content/checkpoint-28032)... Done. 6.7s


## 2. 모델 추론하기

In [ ]:
# 이곳에 내가 사용할 wandb config 설정 "추론에 사용할 ckt 경로 설정"
# loaded_config['inference']['ckt_path'] = "home/ckt"

In [ ]:
# 사용할 device를 정의합니다. GPU가 사용 가능하면 'cuda:0', 그렇지 않으면 'cpu'를 사용합니다.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('-' * 10, f'device : {device}', '-' * 10,)
print(torch.__version__)

### 1) 모델과 토크나이저 불러오기

<font color=red size=32><b> ckt 값 바꾸기

In [ ]:
멈춤

In [ ]:
checkpoint_num = '20254'

In [ ]:
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config, device):
    print('-' * 10, 'Load tokenizer & model', '-' * 10,)

    print("\n######  설정 파일에서 모델 이름과 체크포인트 경로를 불러옵니다.")
    ### 모델 이름과 체크포인트 경로 설정:
    # 설정 파일에서 모델 이름과 체크포인트 경로를 불러옵니다.
    # config['general']['model_name']: 사전 학습된 모델의 이름을 설정 파일에서 가져옵니다.
    # config['inference']['ckt_path']: 학습된 모델의 체크포인트 경로를 설정 파일에서 가져옵니다. 이 경로는 학습이 완료된 후 저장된 모델의 위치를 나타냅니다.
    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']  # 학습된 모델의 체크포인트 경로
    ckt_path = "/home/code/checkpoint-" + checkpoint_num
    print(model_name, " / ", ckt_path)
    print('-' * 10, f'Model Name : {model_name}', '-' * 10,)


    ### 토크나이저 로드 및 특수 토큰 추가:
    # AutoTokenizer.from_pretrained(model_name): 사전 학습된 모델 이름을 사용하여 토크나이저를 로드합니다.
    # tokenizer.add_special_tokens(special_tokens_dict): 설정 파일에서 정의된 특수 토큰을 토크나이저에 추가합니다. 이는 모델이 특정 단어들을 특수 토큰으로 처리할 수 있도록 합니다.
    print("\n######  지정된 모델 이름으로부터 토크나이저를 로드합니다.")
    # 지정된 모델 이름으로부터 토크나이저를 로드합니다.
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("\n######  설정 파일에서 정의된 특수 토큰을 토크나이저에 추가합니다.")
    # 설정 파일에서 정의된 특수 토큰을 토크나이저에 추가합니다.
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)


    ### 모델 로드:
    # BartForConditionalGeneration.from_pretrained(ckt_path): 학습된 모델의 체크포인트에서 BART 모델을 로드합니다.
    # generate_model.resize_token_embeddings(len(tokenizer)): 토크나이저에 추가된 특수 토큰에 맞춰 모델의 토큰 임베딩 크기를 재조정합니다.
    print("\n######  학습된 체크포인트에서 BART 모델을 로드합니다.")
    # 학습된 체크포인트에서 BART 모델을 로드합니다.
    generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
    print("\n######  추가된 토큰에 맞게 모델의 토큰 임베딩 크기를 조정합니다.")
    # 추가된 토큰에 맞게 모델의 토큰 임베딩 크기를 조정합니다.
    generate_model.resize_token_embeddings(len(tokenizer))


    ### 디바이스로 모델 이동:
    # generate_model.to(device): 로드된 모델을 지정된 디바이스(GPU 또는 CPU)로 이동시켜,
    # 추론 작업을 해당 디바이스에서 수행할 수 있도록 합니다.
    print("\n######  모델을 지정된 디바이스(GPU 또는 CPU)로 이동시킵니다.")
    # 모델을 지정된 디바이스(GPU 또는 CPU)로 이동시킵니다.
    generate_model.to(device)

    print('-' * 10, 'Load tokenizer & model complete', '-' * 10,)


    ### 모델과 토크나이저 반환:
    # 최종적으로 불러온 모델과 토크나이저를 반환하여, 이후 추론 작업에서 사용할 수 있도록 합니다.
    return generate_model, tokenizer


In [ ]:
print("\n#######  추론을 위한 모델과 토크나이저를 불러옵니다.  ##############################################################################")
# 추론을 위한 모델과 토크나이저를 불러옵니다.
generate_model, tokenizer = load_tokenizer_and_model_for_test(config, device)

### 2) 데이터 전처리

In [ ]:
print("\n#######  데이터 경로와 전처리기를 설정합니다.  ####################################################################################")
# 데이터 경로와 전처리기를 설정합니다.
data_path = config['general']['data_path']
preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

### 3) 테스트 데이터셋 준비

In [ ]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):

    ### 테스트 데이터 로드:
    # test_file_path에서 test.csv 파일을 읽어옵니다.
    test_file_path = os.path.join(config['general']['data_path'], 'test.csv')
    # Preprocess 클래스의 make_set_as_df 메서드를 사용하여 테스트 데이터를 데이터프레임으로 변환합니다. 이때, is_train=False로 설정하여 학습과 달리 요약(summary)이 포함되지 않은 테스트 데이터를 처리합니다.
    # test_data['fname']는 테스트 데이터의 고유 ID로, 각 샘플을 식별하는 데 사용됩니다.
    # 테스트 데이터를 데이터프레임으로 변환합니다. is_train=False로 설정하여 테스트 데이터셋을 로드합니다.
    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data['fname']  # 테스트 데이터의 고유 ID (fname)을 가져옵니다.


    # 데이터 확인:
    # 테스트 데이터의 첫 번째 대화(dialogue) 샘플을 출력하여, 데이터가 올바르게 로드되었는지 확인합니다.
    print('-' * 150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-' * 150)


    ### 인코더 및 디코더 입력 생성:
    # Preprocess 클래스의 make_input 메서드를 사용하여 인코더 입력과 디코더 입력을 생성합니다.
    # 테스트 데이터의 경우, 디코더 입력은 시작 토큰(bos_token)만 포함합니다.
    # 테스트 데이터에 대해 인코더 입력과 디코더 입력을 생성합니다. is_test=True로 설정하여 디코더 입력에만 시작 토큰을 넣습니다.
    encoder_input_test, decoder_input_test = preprocessor.make_input(test_data, is_test=True)
    print('-' * 10, 'Load data complete', '-' * 10,)


    ### 토큰화:
    # tokenizer를 사용하여 인코더 입력과 디코더 입력을 토큰화합니다.
    # 이 과정에서 패딩, 특수 토큰 추가, 최대 길이 제한 등을 설정하여 텐서 형태로 반환합니다.
    # 테스트 데이터의 인코더 입력을 토크나이저로 토큰화합니다.
    test_tokenized_encoder_inputs = tokenizer(
        encoder_input_test, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,
    )
    # 테스트 데이터의 디코더 입력을 토크나이저로 토큰화합니다.
    test_tokenized_decoder_inputs = tokenizer(
        decoder_input_test, return_tensors="pt", padding=True,
        add_special_tokens=True, truncation=True,
        max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,
    )


    ### 테스트 데이터셋 준비:
    # DatasetForInference 클래스를 사용하여 토큰화된 입력 데이터와 테스트 ID를 포함한 데이터셋 객체를 생성합니다.
    # 이 데이터셋 객체는 모델이 예측을 수행하는 데 사용됩니다.
    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-' * 10, 'Make dataset complete', '-' * 10,)


    ### 결과 반환:
    # 원본 테스트 데이터(test_data)와 모델에 입력될 토큰화된 데이터셋(test_encoder_inputs_dataset)을 반환합니다.
    return test_data, test_encoder_inputs_dataset


    # 이 함수는 테스트 데이터를 전처리하여 모델에 적합한 입력 데이터로 준비합니다.
    # 이를 통해 학습된 모델이 테스트 데이터에 대한 예측을 정확하게 수행할 수 있도록 합니다.



In [ ]:
print("\n#######  테스트 데이터셋을 준비합니다.  ##########################################################################################")
# 테스트 데이터셋을 준비합니다.
test_data, test_encoder_inputs_dataset = prepare_test_dataset(config, preprocessor, tokenizer)


### 4) 데이터 로더 생성 / 테스트 데이터 추론

In [ ]:
print("\n#######  데이터 로더를 생성합니다. 배치 크기는 설정 파일에서 가져옵니다.  ##########################################################")
# 데이터 로더를 생성합니다. 배치 크기는 설정 파일에서 가져옵니다.
dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

summary = []  # 요약문을 저장할 리스트
text_ids = []  # 텍스트 ID를 저장할 리스트

with torch.no_grad():  # 추론 중에는 기울기 계산을 비활성화하여 메모리를 절약합니다.
    for item in tqdm(dataloader):  # 데이터 로더를 통해 배치 단위로 데이터에 접근합니다.
        text_ids.extend(item['ID'])  # 현재 배치의 텍스트 ID를 저장합니다.
        generated_ids = generate_model.generate(
            input_ids=item['input_ids'].to(device),  # 인코더 입력을 디바이스로 이동시켜 모델에 전달합니다.
            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],  # 반복 n-gram 방지 설정
            early_stopping=config['inference']['early_stopping'],  # 조기 종료 설정
            max_length=config['inference']['generate_max_length'],  # 생성할 텍스트의 최대 길이
            num_beams=config['inference']['num_beams'],  # 빔 서치(beam search)의 빔 수 설정
        )
        for ids in generated_ids:  # 생성된 요약문 ID를 디코딩하여 텍스트로 변환합니다.
            result = tokenizer.decode(ids, skip_special_tokens=False)  # 특수 토큰을 건너뛰고 디코딩합니다.
            summary.append(result)  # 생성된 요약문을 리스트에 추가합니다.

### 5) 최종 결과/요약문 전처리

In [ ]:
print("\n#######  스페셜 토큰을 제거하여 최종 요약문을 전처리합니다.  ######################################################################")
# 스페셜 토큰을 제거하여 최종 요약문을 전처리합니다.
remove_tokens = config['inference']['remove_tokens']
preprocessed_summary = summary.copy()
for token in remove_tokens:
    preprocessed_summary = [sentence.replace(token, " ") for sentence in preprocessed_summary]

In [ ]:

for i in range(len(preprocessed_summary)):
    tmp = preprocessed_summary[i]
    left_trimmed = tmp.lstrip()  # 왼쪽 공백 제거
    right_trimmed = left_trimmed.rstrip()  # 오른쪽 공백 제거
    tmp = right_trimmed
    tmp = tmp.replace('# ', '#')
    preprocessed_summary[i] = tmp

In [ ]:
preprocessed_summary

### 6) 최종 결과 데이터프레임으로 정리

In [ ]:
print("\n#######  최종 결과를 데이터프레임으로 정리합니다. ###############################################################################")
# 최종 결과를 데이터프레임으로 정리합니다.
output = pd.DataFrame({
    "fname": test_data['fname'],  # 파일 이름
    "summary": preprocessed_summary,  # 전처리된 요약문
})

### 7) CSV 파일 저장

In [ ]:
print("#######  ", model.replace("/", "-") + para + "-ck" + checkpoint_num + "_output.csv" + "  파일이 완성되었습니다.")

In [ ]:
print("\n#######  결과를 저장할 경로를 설정하고, 결과를 CSV 파일로 저장합니다.  ##########################################################")
# 결과를 저장할 경로를 설정하고, 결과를 CSV 파일로 저장합니다.
result_path = config['inference']['result_path']
if not os.path.exists(result_path):
    os.makedirs(result_path)  # 경로가 존재하지 않으면 디렉토리를 생성합니다.
output.to_csv(os.path.join(result_path, model.replace("/", "-") + para + "-ck" + checkpoint_num + "_output.csv"), index=False)  # 결과를 CSV 파일로 저장합니다.



In [ ]:
# 결과 확인
output